# Pdf Downloading

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import urllib3
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re

In [ ]:

# Suppress only the single InsecureRequestWarning from urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Function to download a file from a URL
def download_file(url, folder):
    local_filename = url.split('/')[-1]
    local_path = os.path.join(folder, local_filename)
    with requests.get(url, stream=True, verify=False) as r:
        r.raise_for_status()
        with open(local_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_path

# Function to scrape PDF links using BeautifulSoup
def scrape_and_download_pdfs(page_url, folder):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(page_url, headers=headers, verify=False)
    if response.status_code != 200:
        print(f"Failed to fetch {page_url}: {response.status_code}")
        return

    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a', href=True)

    # Debug: Print all links
    print(f"Found {len(links)} links on {page_url}")
    for link in links:
        print(f"Text: {link.text.strip()}, Href: {link['href']}")

    # Download PDFs that start with 'summary_' or match the pattern '****report.pdf'
    for link in links:
        href = link['href']
        filename = href.split('/')[-1]
        if (filename.startswith('summary_') and filename.endswith('.pdf')) or re.match(r'^\d{4}report\.pdf$', filename):
            if not href.startswith('http'):
                href = 'https://misc.bpdb.gov.bd' + href
            print(f"Found PDF link: {href}")
            download_file(href, folder)

# Function to scrape PDFs using Selenium (optional for JavaScript-heavy pages)
def scrape_with_selenium(page_url, folder):
    driver = webdriver.Chrome()  # Ensure you have ChromeDriver installed
    driver.get(page_url)
    time.sleep(5)  # Wait for the page to load

    links = driver.find_elements(By.TAG_NAME, 'a')
    for link in links:
        href = link.get_attribute('href')
        filename = href.split('/')[-1] if href else ''
        if href and ((filename.startswith('summary_') and filename.endswith('.pdf')) or re.match(r'^\d{4}report\.pdf$', filename)):
            if not href.startswith('http'):
                href = 'https://misc.bpdb.gov.bd' + href
            print(f"Found PDF link: {href}")
            download_file(href, folder)
    driver.quit()

# Main function to iterate over all pages and download PDFs
def main():
    base_url = 'https://misc.bpdb.gov.bd/daily-generation-archive?page='
    download_folder = r'pdf'  # Set the directory where you want to save the PDFs
    os.makedirs(download_folder, exist_ok=True)

    # Iterate over the pages (adjust range as needed)
    for page_num in range(184, 190):
        page_url = base_url + str(page_num)
        print(f"Scraping page: {page_url}")

        # Use BeautifulSoup for scraping
        scrape_and_download_pdfs(page_url, download_folder)

if __name__ == '__main__':
    main()

# Excel Generation

In [4]:
import os
import pdfplumber
import pandas as pd
import re

In [ ]:
# Function to normalize data keys
def normalize_keys(data):
    key_map = {
        "Max. Demand at eve. peak (Generation end)": "Max. Demand at eve. peak (Generation end)",
        "Max. Demand (Generation end)": "Max. Demand at eve. peak (Generation end)",
        "Max. Demand at eve. peak (Sub-station end)": "Max. Demand at eve. peak (Sub-station end)",
        "Max. Demand (Sub-station end)": "Max. Demand at eve. peak (Sub-station end)",
        "Minimum Generation Forecast up to 8:00 hrs.": "Minimum Generation Forecast up to 8:00 hrs.",
        "Actual Minimum Generation up to 8:00 hrs.": "Minimum Generation Forecast up to 8:00 hrs.",
        "Gas limitation": "Gas/LF limitation",
        "KhMulna_demand": "Khulna_demand",
        "KhMulna_supply": "Khulna_supply",
        "KhMulna_load": "Khulna_load",
        "Maximum Temperature in Dhaka was : 29.6° C": "Maximum Temperature in Dhaka was",
        "Maximum Temperature in Dhaka was : 29.6°C": "Maximum Temperature in Dhaka was",
        "Maximum Temperature in Dhaka was : 31° C": "Maximum Temperature in Dhaka was"
    }
    return {key_map.get(k, k): v for k, v in data.items()}

# Function to extract specific data from the text
def extract_specific_data(text):
    data = {}

    # Extract date and day of the week
    date_match = re.search(r"\(C\) Actual data of (\d{2}\.\d{2}\.\d{2}) \(\w+\) (\w+)", text)
    if date_match:
        data['Actual data of'] = date_match.group(1)
        data['Day of the week'] = date_match.group(2)

    # Extract various generation values using regular expressions
    max_demand_gen_match = re.search(r"(Max\. Demand at eve\. peak \(Generation end\)|Max\. Demand \(Generation end\)) : (\d+\.?\d*) MW", text)
    if max_demand_gen_match:
        data['Max. Demand at eve. peak (Generation end)'] = float(max_demand_gen_match.group(2))

    max_demand_sub_match = re.search(r"(Max\. Demand at eve\. peak \(Sub-station end\)|Max\. Demand \(Sub-station end\)) : (\d+\.?\d*) MW", text)
    if max_demand_sub_match:
        data['Max. Demand at eve. peak (Sub-station end)'] = float(max_demand_sub_match.group(2))

    highest_gen_match = re.search(r"Highest Generation \(Generation end\) : (\d+\.?\d*) MW", text)
    if highest_gen_match:
        data['Highest Generation (Generation end)'] = float(highest_gen_match.group(1))

    min_gen_match = re.search(r"Minimum Generation \(Generation end\) : (\d+\.?\d*) MW", text)
    if min_gen_match:
        data['Minimum Generation (Generation end)'] = float(min_gen_match.group(1))

    day_peak_gen_match = re.search(r"Day-peak Generation \(Generation end\) : (\d+\.?\d*) MW", text)
    if day_peak_gen_match:
        data['Day-peak Generation (Generation end)'] = float(day_peak_gen_match.group(1))

    evening_peak_gen_match = re.search(r"Evening-peak Generation \(Generation end\) : (\d+\.?\d*) MW", text)
    if evening_peak_gen_match:
        data['Evening-peak Generation (Generation end)'] = float(evening_peak_gen_match.group(1))

    min_gen_forecast_match = re.search(r"(Minimum Generation Forecast up to 8:00 hrs\.|Actual Minimum Generation up to 8:00 hrs\.) : (\d+\.?\d*) MW", text)
    if min_gen_forecast_match:
        data['Minimum Generation Forecast up to 8:00 hrs.'] = float(min_gen_forecast_match.group(2))

    # Extract temperature with different formats
    max_temp_match = re.search(r"Maximum Temperature in Dhaka was : (\d+\.?\d*)° ?C", text)
    if max_temp_match:
        data['Maximum Temperature in Dhaka was'] = float(max_temp_match.group(1))

    gas_limitation_match = re.search(r"(Gas/LF limitation|Gas limitation) : (\d+) MW", text)
    if gas_limitation_match:
        data['Gas/LF limitation'] = int(gas_limitation_match.group(2))

    coal_supply_limitation_match = re.search(r"Coal supply Limitation : (\d+) MW", text)
    if coal_supply_limitation_match:
        data['Coal supply Limitation'] = int(coal_supply_limitation_match.group(1))

    water_level_limitation_match = re.search(r"Low water level in Kaptai lake : (\d+) MW", text)
    if water_level_limitation_match:
        data['Low water level in Kaptai lake'] = int(water_level_limitation_match.group(1))

    plants_shutdown_match = re.search(r"Plants under shut down/ maintenance : (\d+) MW", text)
    if plants_shutdown_match:
        data['Plants under shut down/ maintenance'] = int(plants_shutdown_match.group(1))

    # Extract demand, supply, and load shed for each zone
    zones = ['Dhaka', 'Chattogram', 'Khulna', 'Rajshahi', 'Mymensingh', 'Sylhet', 'Barishal', 'Rangpur', 'Cumilla']
    for zone in zones:
        zone_match = re.search(rf"{zone} (\d+) (\d+) (\d+)", text)
        if zone_match:
            data[f'{zone}_demand'] = int(zone_match.group(1))
            data[f'{zone}_supply'] = int(zone_match.group(2))
            data[f'{zone}_load'] = int(zone_match.group(3))

    # Handle the case where Khulna is misspelled as KhMulna
    zone_match = re.search(r"KhMulna (\d+) (\d+) (\d+)", text)
    if zone_match:
        data['Khulna_demand'] = int(zone_match.group(1))
        data['Khulna_supply'] = int(zone_match.group(2))
        data['Khulna_load'] = int(zone_match.group(3))

    return normalize_keys(data)

# Directory containing PDF files
pdf_dir = r"pdf" #set the directory where the PDFs are stored

# Path to the Excel file
excel_path = r"data.xlsx" #set the path where you want to save the Excel file

# Path to the error log file
error_log_path = r"errors.txt" #set the path where you want to save the error log file

# Predefine the expected columns to ensure they are always present
expected_columns = [
    "Actual data of", "Day of the week", "Max. Demand at eve. peak (Generation end)",
    "Max. Demand at eve. peak (Sub-station end)", "Highest Generation (Generation end)",
    "Minimum Generation (Generation end)", "Day-peak Generation (Generation end)",
    "Evening-peak Generation (Generation end)", "Minimum Generation Forecast up to 8:00 hrs.",
    "Maximum Temperature in Dhaka was", "Gas/LF limitation", "Coal supply Limitation", 
    "Low water level in Kaptai lake", "Plants under shut down/ maintenance"
]
zones = ['Dhaka', 'Chattogram', 'Khulna', 'Rajshahi', 'Mymensingh', 'Sylhet', 'Barishal', 'Rangpur', 'Cumilla']
for zone in zones:
    expected_columns.extend([f'{zone}_demand', f'{zone}_supply', f'{zone}_load'])

# Initialize the error log
with open(error_log_path, 'w') as error_log:
    error_log.write("Error Log\n")

# Process each PDF file one by one
for filename in os.listdir(pdf_dir):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)
        try:
            with pdfplumber.open(pdf_path) as pdf:
                # Extract text from page 2 (pages are 0-indexed, so page 2 is index 1)
                page = pdf.pages[1]  # Page 2 corresponds to index 1
                text = page.extract_text()
                # Extract specific data from the text
                data = extract_specific_data(text)
                
                # Ensure all expected columns are present
                for col in expected_columns:
                    if col not in data:
                        data[col] = None

                # Convert the extracted data into a DataFrame
                df = pd.DataFrame([data])

                # Append data to the Excel file
                if os.path.exists(excel_path):
                    existing_df = pd.read_excel(excel_path)
                    combined_df = pd.concat([existing_df, df], ignore_index=True)
                    combined_df.to_excel(excel_path, index=False)
                else:
                    df.to_excel(excel_path, index=False)

                print(f"Data from {filename} has been successfully extracted and saved to the Excel file.")

        except Exception as e:
            # Log the error and continue with the next file
            with open(error_log_path, 'a') as error_log:
                error_log.write(f"Error processing {filename}: {str(e)}\n")
            print(f"Error processing {filename}: {str(e)}")

print(f"Data extraction and saving completed. Errors, if any, have been logged to {error_log_path}.")